In [2]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

In [3]:
#importing all csv files
domarar = pd.read_csv('csv/blak-domarar.csv', sep=';', header=0)
einstaklingar = pd.read_csv('csv/blak-einstaklingar.csv', sep=';', header=0)
forsvarsmenn = pd.read_csv('csv/blak-forsvarsmenn.csv', sep=';', header=0)
lid = pd.read_csv('csv/blak-lid.csv', sep=';', header=0)
lidimoti = pd.read_csv('csv/blak-lidimoti.csv', sep=';', header=0)
lidsmenn = pd.read_csv('csv/blak-lidsmenn.csv', sep=';', header=0)
lidsstjorar = pd.read_csv('csv/blak-lidsstjorar.csv', sep=';', header=0)
thjalfarar = pd.read_csv('csv/blak-thjalfarar.csv', sep=';', header=0)
mot = pd.read_csv('csv/blak-mot.csv', sep=';', header=0)

In [4]:
# drop all SyndarLids with an ID (SyndarlidID)
# (the reason for not dropping using SyndarLid is because I don't trust that column to be inserted correctly with [0,1])
lid = lid[lid['SyndarlidID'].isna()]
# then dropping those two columns because we don't want virtual teams
lid = lid.drop(columns=['SyndarLid', 'SyndarlidID'])

In [5]:
#print(help(einstaklingar))
#print(domarar)
#print(einstaklingar)
#print(forsvarsmenn)
print(lid.columns)
#print(lidimoti)
#print(lidsmenn)
#print(lidsstjorar)
#print(mot)
#print(thjalfarar)

Index(['LidID', 'Nafn', 'Radnumer', 'LidaTegundID', 'Timastimpill'], dtype='object')


In [6]:
#print(domarar.describe())
#print(einstaklingar.describe())
#print(forsvarsmenn.describe())
#print(lid.describe())
#print(lidimoti.describe())
#print(lidsmenn.describe())
#print(lidsstjorar.describe())
#print(mot.describe())
#print(thjalfarar.describe())

In [7]:
#domarar.dtypes
einstaklingar.dtypes
#forsvarsmenn.dtypes
#lid.dtypes
#lidimoti.dtypes
#lidsmenn.dtypes
#lidsstjorar.dtypes
#mot.dtypes
#thjalfarar.dtypes

EinstID            int64
Nafn              object
Fdagur            object
Kyn               object
FelagISI          object
Netfang           object
Heimilisfang1    float64
Heimilisfang2    float64
Heimilisfang3    float64
Simi1             object
Simi2             object
Simi3             object
Timastimpill      object
Haed             float64
dtype: object

In [8]:
#make it easier to copy data
#domarar
#einstaklingar
#forsvarsmenn
#lidimoti
#lidsmenn
#lidsstjorar
#mot
#thjalfarar

#lid_nafn_valuecount = lid['Nafn'].value_counts()
#print(lid_nafn_valuecount)
#lid_nafn_valuecount[lid_nafn_valuecount < 2]
#lid['LidID'].value_counts() 

#lid_nafn_valuecount2 = lid['Nafn'].value_counts()
#lid_nafn_valuecount2


In [9]:
# All duplicated birthdays
#pd.concat(g for _, g in einstaklingar.groupby("Fdagur") if len(g) > 1)
duplicated_einstaklingar = einstaklingar[einstaklingar.duplicated(subset=['Nafn', 'Fdagur', 'Kyn'], keep=False)]

In [10]:
duplicated_fdagur_kyn_einstaklingar = einstaklingar[einstaklingar.duplicated(subset=['Fdagur', 'Kyn'], keep=False)]

In [18]:
print(duplicated_fdagur_kyn_einstaklingar)

,EinstID,Nafn,Fdagur,Kyn,FelagISI,Netfang,Heimilisfang1,Heimilisfang2,Heimilisfang3,Simi1,Simi2,Simi3,Timastimpill,Haed
6,2930,Aðalbjörn Hannesson,1990-01-01 00:00:00.000,kk,NaN,26aha@ma.is,NaN,NaN,NaN,6916456,NaN,NaN,2010-02-19 17:24:51.373,NaN
8,60,Aðalheiður Ásmunds,1964-03-04 00:00:00.000,kvk,HK,alla@frodi.is,NaN,NaN,NaN,8963021,NaN,NaN,2000-04-28 15:00:23.000,0.0
13,1400,Aðalheiður Sigursveinsdóttir,1973-10-20 00:00:00.000,kvk,HK,asigursveins@hotmail.com,NaN,NaN,NaN,7707222,554-7222,NaN,2003-04-14 22:57:08.747,0.0
14,1348,Aðalheiður Stefánsdóttir,1968-05-15 00:00:00.000,kvk,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2003-03-31 22:02:18.437,NaN
17,1964,Aðalsteinn Eymundsson,1981-03-03 00:00:00.000,kk,NaN,NaN,NaN,NaN,NaN,8212554,NaN,NaN,2006-01-09 20:31:04.420,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
4485,1413,Ölver Þórarinsson,1980-01-01 00:00:00.000,kk,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2003-04-23 14:56:52.873,NaN
4490,1840,Örnólfur Ásmundsson,1954-05-02 00:00:00.000,kk,NaN,NaN,NaN,NaN,NaN,NaN,4671269,NaN,2005-04-12 12:23:56.500,NaN
4491,1841,Örnólfur Ásmundsson,1954-05-02 00:00:00.000,kk,NaN,NaN,NaN,NaN,NaN,NaN,4671269,NaN,2005-04-12 13:56:15.013,NaN
4492,2854,Örnólfur Oddsson,1956-05-26 00:00:00.000,kk,Þróttur,oddi@itn.is,NaN,NaN,NaN,6961188,5577346,5698808,2009-04-26 23:16:29.827,176.0


In [45]:
'''
    if first_name in duplicate_dict:
        duplicate_dict[first_name].append(row)
    else:
        duplicate_dict[first_name] = [row]
'''
#duplicated_fdagur_kyn_einstaklingar["FyrstaNafn"]
for index, row in duplicated_fdagur_kyn_einstaklingar.iterrows():
    full_name = row['Nafn']
    first_name = full_name.split()[0]
    Fdagur_date = row['Fdagur'].split()[0]
    print(first_name, Fdagur_date)


Aðalbjörn 1990-01-01
Aðalheiður 1964-03-04
Aðalheiður 1973-10-20
Aðalheiður 1968-05-15
Aðalsteinn 1981-03-03
Aðalsteinn 1981-03-03
Agata 1979-03-16
Agla 1972-02-25
Agnes 1961-06-22
Agnes 1990-01-01
Agnieszka 1979-08-03
aldís 1996-06-22
aldís 1996-06-22
Aldís 1990-06-19
Alexander 1968-01-01
Alexander 1990-12-05
Alexander 1990-12-05
Alice 1968-02-14
Alma 1961-03-05
Alma 1961-03-05
Alma 1964-10-09
Alma 1964-10-09
Amid 1964-05-05
Ana 1972-01-29
Ana 1972-01-29
Andrea 1992-01-01
Andrea 1968-11-14
Andrei 1999-01-01
Andrés 1967-09-16
Andrés 1967-09-16
Andri 1979-04-10
Andri 1979-04-10
Andri 1989-05-23
Andri 1989-05-23
Andy 1991-01-01
Ane 1990-01-01
Angela 1975-11-19
Anna 1947-10-21
Anna 1977-05-03
Anna 1970-12-26
Anna 1970-12-26
Anna 1975-06-26
ANNA 1972-05-30
ANNA 1972-05-30
anna 1972-05-30
Anna 1972-05-30
Anna 1972-05-30
anna 1972-05-30
anna 1972-05-30
Anna 1978-01-01
Anna 1957-04-07
Anna 1978-05-03
Anna 1978-05-03
Anna 1969-10-02
Anna 1969-10-02
Anna 1969-08-17
Anna 1957-02-23
Anna 1970-09-

Ingibjörg 1962-02-24
Ingibjörg 1967-12-19
Ingibjörg 1974-11-14
Ingibjörg 1978-05-03
Ingibjörg 1961-06-18
Ingibjörg 1961-06-18
Ingibjörg 1969-10-02
Ingibjörg 1969-10-02
Ingibjörg 1976-12-25
Ingibjörg 1971-07-12
Ingólfur 1967-07-12
Ingólfur 1967-07-12
Ingólfur 1991-02-08
Ingólfur 1991-02-08
Ingólfur 1980-05-14
Ingrid 1992-01-01
Ingunn 1991-01-01
Ingunn 1966-03-12
Ingunn 1966-03-12
Ingunn 1967-06-20
Ingvar 1996-06-29
Ingveldur 1965-04-09
Ingveldur 1965-04-09
Ingveldur 1965-04-09
Irene 1978-08-26
Irene 1978-08-26
Iris 1967-10-03
Ismar 1978-03-26
Ismar 1978-03-26
Ivo 1974-01-01
Ivo 1974-01-01
Ivo 1974-01-01
Ivo 1974-01-01
Íris 1997-07-15
Íris 1975-01-29
Íris 1976-08-13
Íris 1968-03-08
Íris 1968-03-08
Íris 1979-07-27
Íris 1967-10-03
Íris 1979-07-27
Ísak 1997-05-14
Jakob 1991-12-23
Jakob 1971-07-31
Jason 1900-01-01
Jeanette 2008-01-01
Jeff 1970-01-01
Jeff 1970-01-01
Jens 1969-12-03
Jens 1969-12-03
JGSoJGT 1930-01-01
Jim 1970-01-01
Johann 1974-08-03
John 1971-07-06
Jon 1973-04-26
Josefin 1991-

Somjai 1974-02-05
Sóley 1964-01-27
Sóley 1964-01-27
Sóley 1980-06-17
Sólrún 1978-08-26
Sólrún 1964-05-05
Sólrún 1978-06-27
Sólveig 1965-02-15
Sólveig 1974-11-17
Sólveig 1974-11-17
Sólveig 1974-11-17
Sólveig 1965-02-15
Sólveig 1965-02-15
sólveig 1965-02-15
Sólveig 1944-04-17
Sólveig 1944-04-17
Sólveig 1954-07-27
Sólveig 1954-07-27
Stefanía 1967-04-03
Stefán 1953-02-05
Stefán 1953-02-05
Stefán 1980-09-12
Stefán 1980-09-12
Stefán 1980-09-12
Stefán 1972-12-11
Stefán 1972-12-11
steinar 1971-08-25
Steinar 1965-09-05
steinar 1971-08-25
Steinar 1971-08-25
Steindóra 1972-03-03
Steindóra 1972-03-03
Steini 1959-08-18
Steinn 1970-05-17
Steinn 1970-05-17
Steinunn 1996-02-15
Steinunn 1976-01-28
Steinunn 1964-05-02
Steinunn 1969-04-24
Steinunn 1969-04-24
Steinunn 1969-04-24
Steinþór 1976-06-21
Støyten 1990-07-08
Støyten 1990-07-08
Sunna 1968-03-08
Sunna 1994-05-29
Sunna 1990-05-29
Sunna 1990-05-29
Sunna 1994-05-29
Susan 1964-08-23
Susan 1964-08-23
Svala 1961-06-17
Svala 1961-06-17
Svala 1961-06-17
Sv

In [ ]:
duplicate_dict

In [ ]:
## change all other table entries to the main id for new person
## delete old values ???

In [153]:
#make it easier to copy data
#domarar
#einstaklingar
#forsvarsmenn
#lid
#lidimoti
#lidsmenn
#lidsstjorar
#mot
#thjalfarar

In [187]:
#FINAL STEP (run after everything is done):

#duplicated people put into it's own csv to be browsed later
pd.DataFrame(duplicated_einstaklingar).to_csv("csv/new/duplicated-einstaklingar.csv", encoding='utf-8-sig')

#save as new csv inside csv/new
pd.DataFrame(domarar).to_csv("csv/new/blak-domarar.csv", encoding='utf-8-sig')
pd.DataFrame(einstaklingar).to_csv("csv/new/blak-einstaklingar.csv", encoding='utf-8-sig')
pd.DataFrame(forsvarsmenn).to_csv("csv/new/blak-forsvarsmenn.csv.csv", encoding='utf-8-sig')
pd.DataFrame(lid).to_csv("csv/new/blak-lid.csv", encoding='utf-8-sig')
pd.DataFrame(lidimoti).to_csv("csv/new/blak-lidimoti.csv", encoding='utf-8-sig')
pd.DataFrame(lidsmenn).to_csv("csv/new/blak-lidsmenn.csv", encoding='utf-8-sig')
pd.DataFrame(lidsstjorar).to_csv("csv/new/blak-lidsstjorar.csv", encoding='utf-8-sig')
pd.DataFrame(mot).to_csv("csv/new/blak-mot.csv", encoding='utf-8-sig')
pd.DataFrame(thjalfarar).to_csv("csv/new/blak-thjalfarar.csv", encoding='utf-8-sig')